In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
renatosn_sao_paulo_housing_prices_path = kagglehub.dataset_download('renatosn/sao-paulo-housing-prices')

print('Data source import complete.')


# Análise Diagnóstica do preço do aluguel em São Paulo

**setup inicial**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



# Explorando o Dataset
Aqui vou analisar o dataset usando pandas

**Primeiro vou ler o csv e verificar as linhas e colunas**

In [ ]:
df = pd.read_csv('/kaggle/input/sao-paulo-housing-prices/data.csv')
print(df.shape)
print("\n")
print(df.columns)

**vendo o formato do df**

In [ ]:
df.head()

**uma descrição estatística**

In [ ]:
df.describe(include = 'all')

# Analisando o describe:


* Os 4 tipos são: kitnet, apartamento, casa_em_condominio e casa.
* Média consideravelmente maior que a mediana e valores extremos no max de área, rent e total indicam uma distribuição com cauda a direita, portanto será necessário cuidar desses outliers.
* 0 não pode ser o mínimo de área (bedroom e garage podem).


quanto aos outliers será necessário:
* um limite inferior por regra de negócio para area
* limites superiores para area, bedrooms, garage e rent. Evitando, portanto a inclusão de imóveis de luxo.

# Manipulação do dataset
Será feita uma verificação e transformação do dataset para que possamos proseguir para a próxima etapa.

Abaixo vou verificar os **tipos de valores** e se há algum **valor nulo** ou **infinito** no dataset (mesmo que seja improvável). Vou verificar também por **duplicatas**.

In [ ]:
print(df.info())  # tipos de dados + contagem de não nulos
print('\n')
print(df.isnull().sum())  # quantidade de valores ausentes por coluna
print('\n')
print("Tem infinitos?", np.isinf(df.select_dtypes(include=[np.number])).values.sum() > 0) # possui inf ou -inf?
print('\n')
print("Possui ",df.duplicated().sum()," Duplicatas") #verificando se os dados estão sem duplicatas

**Tratamento de Outliers**

removendo outliers por regra de negócio:

In [ ]:
# Para 'area'
limite_inferior_area = 30
limite_superior_area = 150
print(limite_inferior_area)
print(limite_superior_area)

# Para 'bedroom'
limite_inferior_bedrooms = 0
limite_superior_bedrooms = 4
print(limite_inferior_bedrooms)
print(limite_superior_bedrooms)

# Para 'garage'
limite_inferior_garage = 0
limite_superior_garage = 2
print(limite_inferior_garage)
print(limite_superior_garage)

# Para 'total'
limite_inferior_total = 1000
limite_superior_total = 8000
print(limite_inferior_total)
print(limite_superior_total)

df_limpo = df[
    (df['area'] >= limite_inferior_area) & (df['area'] <= limite_superior_area) &
    (df['garage'] >= limite_inferior_garage) & (df['garage'] <= limite_superior_garage) &
    (df['bedrooms'] >= limite_inferior_bedrooms) & (df['bedrooms'] <= limite_superior_bedrooms) &
    (df['total'] >= limite_inferior_total) & (df['total'] <= limite_superior_total)
]


print(f"Antes: {df.shape[0]} linhas")
print(f"Removidas: {df.shape[0] - df_limpo.shape[0]} linhas")
print(f"Depois: {df_limpo.shape[0]} linhas")

removendo 'rent' pois o valor que nos importa é 'total'

In [ ]:
df_limpo = df_limpo.drop(columns = 'rent')


Verificando novamente as estatísticas descritivas

In [ ]:
df_limpo.shape

In [ ]:
df_limpo.describe(include = 'all')

visualizando o formato novamente

In [ ]:
df_limpo.head()

# Enriquecimento dos dados

* o total será substituido pelo preço por metro quadrado.

* o type será transformado com one-hot encoding para analisar cada tipo separadamente.

* o distric e address vão ser usados para categorização por preço médio da região e vou criar rótulos com 5 categorias sendo 0 áreas com a média do m² barata e 5 as mais caras.


In [ ]:
#reformulação do total
df_limpo.total = round(df_limpo.total / df_limpo.area, 2)
df_limpo.rename(columns={'total': 'cost/m²'}, inplace = 'true')

In [ ]:
df_limpo = pd.get_dummies(df_limpo, columns=['type'])

boolean_columns = ['type_Apartamento','type_Casa','type_Casa em condomínio','type_Studio e kitnet']

# Converter as colunas para int
for col in boolean_columns:
    df_limpo[col] = df_limpo[col].astype(int)


In [ ]:
media_por_endereço = df_limpo.groupby('address')['cost/m²'].mean().reset_index()
media_por_endereço.columns = ['address', 'avg_price_per_m2']
media_por_endereço['valor_regiao'] = pd.cut(media_por_endereço['avg_price_per_m2'],bins=5,labels=[0,1,2,3,4],
include_lowest=True)
df_limpo = df_limpo.merge(media_por_endereço[['address', 'valor_regiao']], on='address', how='left')
df_limpo = df_limpo.drop(columns = ['address', 'district'])
df_limpo['valor_regiao'] = df_limpo['valor_regiao'].astype(int)

df_limpo.head()

# Análise exploratória dos dados limpos

gráficos para EDA

In [ ]:
import matplotlib.pyplot as plt
df_limpo['cost/m²'].hist(bins=100)
plt.title("Distribuição de preços")
plt.xlabel("R$/m²")
plt.ylabel("Quantidade de imóveis")
plt.show()

df_limpo['area'].hist(bins=10)
plt.title("Distribuição de area")
plt.xlabel("m²")
plt.ylabel("Quantidade de imóveis")
plt.show()

# Configurar tamanho do gráfico
plt.figure(figsize=(14, 6))

# Criar boxplot
sns.boxplot(data=df_limpo, x='valor_regiao', y='cost/m²')

# Melhorar legibilidade
plt.xticks(rotation=90)
plt.title('Distribuição do Preço por classificação de região')
plt.ylabel('R$/m²')
plt.xlabel('valor_regiao')
plt.tight_layout()
plt.show()

sns.boxplot(data=df_limpo, x='type_Apartamento', y='cost/m²')

plt.xticks(rotation=90)
plt.title('Distribuição do Preço por tipo')
plt.ylabel('R$/m²')
plt.xlabel('Apartamento')
plt.tight_layout()
plt.show()

sns.boxplot(data=df_limpo, x='type_Casa', y='cost/m²')

plt.xticks(rotation=90)
plt.title('Distribuição do Preço por tipo')
plt.ylabel('R$/m²')
plt.xlabel('Casa')
plt.tight_layout()
plt.show()

sns.boxplot(data=df_limpo, x='type_Casa em condomínio', y='cost/m²')

plt.xticks(rotation=90)
plt.title('Distribuição do Preço por tipo')
plt.ylabel('R$/m²')
plt.xlabel('Casa em condomínio')
plt.tight_layout()
plt.show()

sns.boxplot(data=df_limpo, x='type_Studio e kitnet', y='cost/m²')

plt.xticks(rotation=90)
plt.title('Distribuição do Preço por tipo')
plt.ylabel('R$/m²')
plt.xlabel('Studio e kitnet')
plt.tight_layout()
plt.show()

sns.scatterplot(data=df_limpo, x='area', y='cost/m²', s=10)
plt.show()

sns.heatmap(df_limpo.corr())
print(df_limpo.corr())

In [ ]:
df_limpo.describe()


Hipóteses a serem testadas em ordem de influência:
* o valor da região é o fator mais determinante pois se relaciona com a criminalidade, proximidade de comércio entre outros fatores derivados da geolocalização.
* o preço por metro quadrado é inversamente proporcional a area.
* quanto mais quartos menos costuma de pagar por metro quadrado
* quanto mais garagens mais caro costuma se pagar por metro quadrado

A classificação do tipo de imóvel perde sua relevância aqui já que na análise gráfica já entendemos suas distribuições e em critério de determinancia as outras features são mais relevantes.

# Modelagem Diagnóstica

Aqui aplico machine learning (random forest regressor) para testar as hipóteses feitas com base na análise exploratória e portanto "minerar" os dados, descobrindo as features mais relevantes.
Não utilizei o type como feature pois após um teste determinei que ele não é um fator de grande relevância.
Talvez com outras features derivadas da localização, como proximidade do metro, criminalidade ou mesmo classificação de pagar ou não pagar condomínio uma análise mais rica pudesse ser feita.

Separando as features da predição e fazendo o split do dataset entre treino e teste.

In [ ]:
from sklearn.model_selection import train_test_split
X = df_limpo[['area', 'bedrooms', 'garage', 'valor_regiao']]
Y = df_limpo[['cost/m²']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)

aplicando e avaliando o modelo random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, Y_train)
y_pred_rf = rf_model.predict(X_test)

mse_rf = mean_squared_error(Y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(Y_test, y_pred_rf)

print("\n--- RandomForestRegressor ---")
print(f"MSE: {mse_rf:.2f}")
print(f"RMSE: {rmse_rf:.2f}")
print(f"R²: {r2_rf:.2f}")
print('\n')

# Interpretabilidade: Feature Importance
print("Importância das Features (Random Forest):")
feature_importances = pd.Series(rf_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feature_importances)

# Exportando os dados
exportando os dados pós mineração para elaborar um dashboard e uma apresentação clara sobre os dados em relação ao negócio (mercado imobiliário de aluguel em São Paulo).
O objetivo é responder quais, como, e porque (com análise de BI), os fatores mais determinantes no preço do m²/mês em São Paulo, realizando assim uma análise diagnostica que pode ser usada para predição e prescrição em outros casos.

In [ ]:
df_limpo.to_csv('dados_tratados.csv', index=False, encoding='utf-8-sig')

# Comentários finais
Pontos a melhorar:
* integração direta do código com a base (quinto andar) usando o webscrapping, ao invés da base do kaggle, melhorando a confiabilidade com dados em tempo real.
* feature enginnering relativo ao tipo de imóvel e principalmente a localização, o que só seria possível com uma coleta mais rica.

Insights:
Todo o processo de dados tem como objetivo extrair conhecimento e enriquecer a sabedoria de negócio, mas para isso cada etapa é crucial, é necessário muita cautela e revisões para garantir um fluxo de trabalho o mais linear possível evitando retomadas para etapas anteriores.
Além disso, o processo de dados sempre pode ter seu valor aumentado.
Quanto mais próxima da arquitetura do processo e mais distante da análise final, mais crítica é a etapa, pois erros na base tendem a ser notados só posteriormente encarecendo suas correções.
